### Importing Libraries

In [ ]:
!pip install opencv-contrib-python==3.4.2.17
!pip install --upgrade imutils

     |████████████████████████████████| 30.6MB 138kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
Requirement already up-to-date: imutils in /usr/local/lib/python3.6/dist-packages (0.5.3)


In [ ]:
import cv2
import numpy as np
import imutils
import matplotlib.pyplot as plt
from random import randrange
from google.colab.patches import cv2_imshow
import sys
import time
import os
import tqdm
from moviepy.editor import ImageSequenceClip

# For 24->8bit
from PIL import Image

%matplotlib inline

For Stitcher

In [ ]:
def cylindricalWarp(img, K):
    """This function returns the cylindrical warp for a given image and intrinsics matrix K"""
    h_,w_ = img.shape[:2]
    # pixel coordinates
    y_i, x_i = np.indices((h_,w_))
    X = np.stack([x_i,y_i,np.ones_like(x_i)],axis=-1).reshape(h_*w_,3) # to homog
    Kinv = np.linalg.inv(K) 
    X = Kinv.dot(X.T).T # normalized coords

    # calculate cylindrical coords (sin\theta, h, cos\theta)
    A = np.stack([np.sin(X[:,0]),X[:,1],np.cos(X[:,0])],axis=-1).reshape(w_*h_,3)
    B = K.dot(A.T).T # project back to image-pixels plane
    
    # back from homog coords
    B = B[:,:-1] / B[:,[-1]]
    
    # make sure warp coords only within image bounds
    B[(B[:,0] < 0) | (B[:,0] >= w_) | (B[:,1] < 0) | (B[:,1] >= h_)] = -1
    B = B.reshape(h_,w_,-1)
    
    # for transparent borders...
    img_rgba = cv2.cvtColor(img,cv2.COLOR_BGR2BGRA) 
    
    # warp the image according to cylindrical coords
    return cv2.remap(img_rgba, B[:,:,0].astype(np.float32), B[:,:,1].astype(np.float32), cv2.INTER_AREA, borderMode=cv2.BORDER_DEFAULT)

In [ ]:
# To crop the black part of the stitched image

def trim(frame):
  # Cropping the top
  if not np.sum(frame[0]):
    return trim(frame[1:])

  if not np.sum(frame[-1]):
    return trim(frame[:-2])

  if not np.sum(frame[:, 0]):
    return trim(frame[:, 1:])

  if not np.sum(frame[:, -1]):
    return trim(frame[:, :-2])

  return frame

In [ ]:
def trim_cyl(img):
  return img[:, 260:1650]

In [ ]:
def cropping(final_color):
  stitched = final_color.copy()

  # create a 10 pixel border surrounding the stitched image
  # print("Cropping...")
  stitched = cv2.copyMakeBorder(stitched, 10, 10, 10, 10,
    cv2.BORDER_CONSTANT, (0, 0, 0))

  # convert the stitched image to grayscale and threshold it
  # such that all pixels greater than zero are set to 255
  # (foreground) while all others remain 0 (background)
  gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

  ## To get the intermediate results
  # plt.imshow(gray, cmap='gray')
  # plt.title("Gray Image")
  # plt.show()

  # plt.pause(0.2)

  # plt.imshow(thresh, cmap='gray')
  # plt.title("Threshold Image")
  # plt.show()

  # plt.pause(0.2)

  # find all external contours in the threshold image then find
  # the *largest* contour which will be the contour/outline of
  # the stitched image
  cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  c = max(cnts, key=cv2.contourArea)

  # allocate memory for the mask which will contain the
  # rectangular bounding box of the stitched image region
  mask = np.zeros(thresh.shape, dtype="uint8")
  (x, y, w, h) = cv2.boundingRect(c)
  cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

  h = int(0.6*h)

  # plt.imshow(cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1), cmap='gray')
  # plt.title("Rectangular Mask")
  # plt.show()

  # plt.pause(0.2)

  # create two copies of the mask: one to serve as our actual
  # minimum rectangular region and another to serve as a counter
  # for how many pixels need to be removed to form the minimum
  # rectangular region
  minRect = mask.copy()

  # find contours in the minimum rectangular mask and then
  # extract the bounding box (x, y)-coordinates
  cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  c = max(cnts, key=cv2.contourArea)
  (x, y, w, h) = cv2.boundingRect(c)

  # print(x, y, w, h)
  # use the bounding box coordinates to extract the our final
  # stitched image
  stitched_cropped = stitched[25: 25 + 1025, x:x + w]

  # Display the output stitched image to our screen
  # cv2_imshow(stitched_cropped)

  return stitched_cropped

In [ ]:
def stitch(img_1, img_2):
  # img2 = cv2.imread('/content/Screenshot 2020-11-03 112809.png')
  # img3 = cv2.imread('/content/Screenshot 2020-11-03 112821.png')
  
  # img = cv2.cvtColor(img_1,cv2.COLOR_BGR2GRAY)
  # img = cv2.resize(img, (1024, 683))
  # imgm = cv2.cvtColor(img_2,cv2.COLOR_BGR2GRAY)
  # imgm = cv2.resize(imgm, (1024, 683))
  img = img_1.copy()
  imgm = img_2.copy()

  sift = cv2.xfeatures2d.SIFT_create()
  
  # Find the keypoints and descriptors with SIFT
  kp1, des1 = sift.detectAndCompute(imgm,None)
  kp2, des2 = sift.detectAndCompute(img,None)

  bf = cv2.BFMatcher()
  matches = bf.knnMatch(des1,des2, k=2)

  # Apply ratio test
  good = []
  alpha = 0.45
  for m in matches:
      if m[0].distance < alpha*m[1].distance:
          good.append(m)
  matches = np.asarray(good)

  if len(matches[:,0]) >= 7:
      src = np.float32([ kp1[m.queryIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
      dst = np.float32([ kp2[m.trainIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
      H, masked = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)
      
      '''
      Optional
      '''
      # Getting the dimensions of the image (To be used soon)
      # h, w = img.shape

      # pts = np.float32([ [0,0], [0, h-1], [w-1, h-1], [w-1,0 ] ]).reshape(-1,1,2)
      # img__ = cv2.perspectiveTransform(pts, H)

      '''
      To plot a line to display the stitch boundary
      '''
      # img_ = cv2.polylines(imgm, [np.int32(img__)], True, 255, 3, cv2.LINE_AA)
      
      '''
      To print the overlapping image
      '''
      # To reduce the file size
      # plt.imshow(imgm, cmap='gray')
      # cv2_imshow(img_)

  else:
      print("Cannot find enough keypoints.")
      pass

  # Homography Matrix
  # print(H)

  stitched = cv2.warpPerspective(imgm, H, (img.shape[1] + imgm.shape[1], img.shape[0]))
  stitched[0:img.shape[0], 0:img.shape[1]] = img

  stitched = cv2.cvtColor(stitched, cv2.COLOR_BGRA2RGB)

  return stitched

In [ ]:
def stitch_images(img1, img2, img3):
  # Getting the images
  img1 = img1.astype('uint8')
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

  img2 = img2.astype('uint8')
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

  img3 = img3.astype('uint8')
  img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)

  ### Preprocessing
  ## Cylindrical Wrapping
  h, w = img1.shape[:2]
  K = np.array([[800,0,w/2],[0,800,h/2],[0,0,1]]) # mock intrinsics

  # Applying the cylindrical warping
  img_cyl1 = cylindricalWarp(img1, K)
  img_cyl2 = cylindricalWarp(img2, K)
  img_cyl3 = cylindricalWarp(img3, K)

  # Cylindrical Trimming the images
  img_cyl1_crop = trim_cyl(img_cyl1)
  img_cyl2_crop = trim_cyl(img_cyl2)
  img_cyl3_crop = trim_cyl(img_cyl3)

  '''
  **Trying the following:**
  1. Left-Middle
  2. Middle-Right
  3. (1)+(2) - Whole 3 stitched together
  '''

  #### Computing the left-mid
  ### Reversing the middle and left image
  ## Middle
  img_cyl2_crop = cv2.flip(img_cyl2_crop, 1)

  # Left
  img_cyl1_crop = cv2.flip(img_cyl1_crop, 1)

  # Joining Left-Mid
  lm = stitch(img_cyl2_crop, img_cyl1_crop)
  lm = trim(lm)

  # Flipping the result
  lm = cv2.flip(lm, 1)

  #### Computing the mid-right 
  ### Flipping to get middle to original values
  img_cyl2_crop = cv2.flip(img_cyl2_crop, 1)

  # Joining Mid-Right
  mr = stitch(img_cyl2_crop, img_cyl3_crop)
  mr = trim(mr)

  #### Left-Mid + Mid-Right
  final = stitch(lm, mr)
  final = trim(final)

  # Converting to RGB again
  final_color = cv2.cvtColor(final, cv2.COLOR_BGRA2RGB)

  # Cropping the stretched image
  stitched_final = cropping(final_color)

  # Resizing the image
  stitched_cropped_resized = cv2.resize(stitched_final, (1920, 720))

  return stitched_cropped_resized

In [ ]:
def run(left_video, mid_video, right_video):
  
  # Get information about the videos
  n_frames = min( int(left_video.get(cv2.CAP_PROP_FRAME_COUNT)),
                  int(mid_video.get(cv2.CAP_PROP_FRAME_COUNT)),
                  int(right_video.get(cv2.CAP_PROP_FRAME_COUNT)) )
  
  fps = int(left_video.get(cv2.CAP_PROP_FPS))
  frames = []

  count = -1
  for _ in tqdm.tqdm(np.arange(n_frames)):
    
    count += 1

    if (count>-1 and count<1000):
      # Grab the frames from their respective video streams
      ok, left = left_video.read()
      _, mid = mid_video.read()
      _, right = right_video.read()

      if ok:
        # Stitch the frames together to form the panorama
        stitched_frame = stitch_images(left, mid, right)
          
        # No homography could not be computed
        if stitched_frame is None:
          print("[INFO]: Homography could not be computed!")
          continue

        # Add frame to video
        frames.append(stitched_frame)

    elif (count==1000):
      print("Finished Processing!")
      break

    else:
      print("Skipping!")
      
  cv2.destroyAllWindows()
  return frames

In [ ]:
# Defining the paths

left_video_in_path='/content/drive/My Drive/Colab Notebooks/Carla Dataset/City/Left.mp4'
mid_video_in_path='/content/drive/My Drive/Colab Notebooks/Carla Dataset/City/Front.mp4'
right_video_in_path='/content/drive/My Drive/Colab Notebooks/Carla Dataset/City/Right.mp4'

In [ ]:
# Set up video capture
left_video = cv2.VideoCapture(left_video_in_path)
mid_video = cv2.VideoCapture(mid_video_in_path)
right_video = cv2.VideoCapture(right_video_in_path)
print('[INFO]: {}, {} and {} loaded'.format(  left_video_in_path.split('/')[-1],
                                              mid_video_in_path.split('/')[-1],
                                              right_video_in_path.split('/')[-1])  )

print('[INFO]: Image Processing starting....')

# The frames of the stitched images
frames = run(left_video, mid_video, right_video)


  0%|          | 0/14850 [00:00<?, ?it/s]

[INFO]: Left.mp4, Front.mp4 and Right.mp4 loaded
[INFO]: Image Processing starting....



  7%|▋         | 1000/14850 [1:59:26<23:00:20,  5.98s/it]

Finished Processing!


In [ ]:
print('[INFO]: Video stitching finished')

'''Writing via CV2'''
video_out_path='/content/drive/My Drive/Colab Notebooks/Carla Dataset/Images'

for i in range(len(frames)):
  cv2.imwrite(video_out_path+f"/{i}"+".png", frames[i])
  print("Creating : ",video_out_path,f"/{i}",".png")

[INFO]: Video stitching finished
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /0 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /1 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /2 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /3 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /4 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /5 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /6 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /7 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /8 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /9 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /10 .png
Creating :  /content/drive/My Drive/Colab Notebooks/Carla Dataset/Images /1